In [ ]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

## Setup

In [ ]:
df = pd.read_csv('train_and_test.csv', index_col=0)
features = df[df.columns[:54]]
target = df['Cover_Type']

## PCA

In [ ]:
pca_feat = PCA() # this is the PCA object
pca_feat.fit(features) # fit the PCA object on the features that you have
n_comp = 5
pca_n = PCA(n_components=n_comp)
features_transformed_pca = pca_n.fit_transform(features)

## Preparing the data

In [ ]:
np.random.seed(42)
# Separate into training (80%) and testing (20%):
msk = np.random.rand(len(features_transformed_pca)) < 0.8
train_features = features_transformed_pca[msk]
train_target = target[msk]
test_features = features_transformed_pca[~msk]
test_target = target[~msk]

## QDA

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(train_features, train_target)
predicted = qda.predict(test_features)
actual = np.array(test_target)

## QDA Confidence Ellipsoid

## Printing the results

In [ ]:
result = [None]*len(actual)
for i in range(len(actual)):
    result[i] = (predicted[i] == actual[i])
correct_count = [i for i in result if i]
correct_rate = float(len(correct_count))/len(result)
print("The successful prediction rate is: ", correct_rate)
print("The score is: ", qda.score(test_features, test_target))